# Kaggle 내부 경진대회 - 머신러닝 (Except Deep Learning) #

## Planets Dataset ##

In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D


### Load Dataset ###

In [20]:
train = pd.read_csv('D:\CodingData\mljnu\kaggle\ml\datasets\\train_kaggle.csv')
test = pd.read_csv('D:\CodingData\mljnu\kaggle\ml\datasets\\test_kaggle.csv')

In [15]:
type(train)

pandas.core.frame.DataFrame

In [17]:
train.head()

,index,method,number,orbital_period,mass,distance,year
0,0,Radial Velocity,4,241.258000,1.981,13.47,1999
1,1,Transit,1,1.337118,NaN,NaN,2011
2,2,Transit,1,1.573292,NaN,350.00,2012
3,3,Radial Velocity,2,3000.000000,NaN,NaN,2014
4,4,Radial Velocity,1,9.494000,0.026,21.30,2010


In [22]:
test.head()

,index,number,orbital_period,mass,distance,year
0,0,2,74.72000,0.05318,8.82,2011
1,1,1,2.99433,NaN,560.00,2010
2,2,6,10.30390,NaN,613.00,2010
3,3,1,NaN,NaN,91.83,2013
4,4,1,2.46502,NaN,NaN,2014


Dataset을 Train_Data와 Test_Data로 분리해서 주었기 때문에 train_test_split 함수 사용 할 필요 없음  
dataset에 대해 head 속성과 shape속성으로 특성을 확인해봤더니, 대략 Train:test의 Sample 비율이 3:1 정도로 나타남  
  
그리고 Test_Data에는 method Column이 없으므로, Train_Data를 적절한 분류(Classification) 모델로 학습 시킨 뒤,  
각 Test_Data Sample을 알아내는 것을 목표로 잡는다.  

이제 적절한 Train 방법을 채택하기 위해 추가적인 Dataset 분석을 진행한다.  

In [24]:
train.get

<bound method NDFrame.get of      index           method  number  orbital_period     mass  distance  year
0        0  Radial Velocity       4      241.258000  1.98100     13.47  1999
1        1          Transit       1        1.337118      NaN       NaN  2011
2        2          Transit       1        1.573292      NaN    350.00  2012
3        3  Radial Velocity       2     3000.000000      NaN       NaN  2014
4        4  Radial Velocity       1        9.494000  0.02600     21.30  2010
..     ...              ...     ...             ...      ...       ...   ...
746    746  Radial Velocity       1      772.000000  2.70000    127.88  2011
747    747  Radial Velocity       1      829.000000  0.80000     32.70  2001
748    748  Radial Velocity       1        3.092500  0.95000     27.05  2000
749    749          Transit       2       27.402000      NaN       NaN  2012
750    750  Radial Velocity       2       74.720000  0.05318      8.82  2011

[751 rows x 7 columns]>

method column의 class가 총 3개임을 확인,  
  
문제 파악을 진행,  

Supervised Learning(지도학습)  
      Classification(분류)  
          Multi Class Clasification(다중 클래스 분류)
              OvO(일대일 방식)
              OvR(일대다 방식)
                  ```python
                  Class == [Radil Velocity, Trainsit, others]
                  lable == train["method"]
                  ```

In [25]:
# 결측치 확인(NaN Check)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751 entries, 0 to 750
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           751 non-null    int64  
 1   method          751 non-null    object 
 2   number          751 non-null    int64  
 3   orbital_period  725 non-null    float64
 4   mass            376 non-null    float64
 5   distance        588 non-null    float64
 6   year            751 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 41.2+ KB


Train_Data에 대해 Column 3, 4, 5에 결측치를 발견.  
세 Column은 모두 수치형 데이터를 포함하는 Column이다.  
  
  

PipeLine 설계
case 1: 결측치 수가 많지 않은 3번 Column에 대해 중간값으로 처리 ```python .medean() ```  
        결측치 수가 많은 4번 Column에 대해 ```python .dropna(axis = 1) ``` 진행하고 싶지만, 행성 발견 방법에서 질량(Mass)는 매우 중요한 특성이므로,  column 3에 대한 회귀(Regression) 진행